There are quite a lot options in `TFLiteConverter`. Before introducing them in detail, let's classify them into different groups first.

There are generally four types of options, which can be listed as below.
- Debugging options
    The options used for debugging purposes.
- Graph options
    The options that affect the compute graph.
- Quantization-related options
    The options that relates to quantization. They will be discussed in the [quantization](../quantization) tutorials.

First, we will go through the debugging options.

## Debugging options

### dump_jit_model_path
dump_jit_model_path (typing.Optional[str]): The path for dumping the jit model. Defaults to None

When you specify `dump_jit_model_path`, the TorchScript model will be saved to the given file path before converting to TFLite.

### dump_dummy_input_path
dump_dummy_input_path (typing.Optional[str]): The path for dumping the dummy input. Defaults to None

When you specify `dump_dummy_input_path`, the inputs to the model will be saved as the NumPy zipped format `*.npz` to the given file path before converting to TFLite.

### dump_config_path
dump_config_path (typing.Optional[str]): The path for dumping the json config. Defaults to None

When you specify `dump_config_path`, the configuration of the converter will be saved as the JSON format to the given file path before converting to TFLite. Please make sure 
`dump_jit_model_path` is also specified.

### preserve_tensors
preserve_tensors (bool): Preserve the copies of the intermediate tensors. Defaults to False

When you specify `preserve_tensors`, all the intermediate tensors will be preserved, so that you may get the values through `converter.get_output(name)`. This option is used when performing layerwise comparison. For more details, please refer to [the example code](../../examples/converter/convert_and_compare.py).

Next, we will take about the graph options.

## Graph options

### nchw_transpose
nchw_transpose (bool): Whether to perform nchw->nhwc transposes on input and output tensors. When `False` is specified, the arguments `input_transpose` and `output_transpose` will be ignored. Defaults to True

### input_transpose
input_transpose (typing.Optional[typing.Union[bool, typing.Iterable[bool]]], optional): Whether to transpose the input(s). Defaults to None(True for 4d-input, False otherwise).

Note: `input_transpose` is ignored if `nchw_transpose=False`

### output_transpose
output_transpose (typing.Optional[typing.Union[bool, typing.Iterable[bool]]], optional): Whether to transpose the output(s). Defaults to None(True for 4d-input, False otherwise).

Note: `output_transpose` is ignored if `nchw_transpose=False`

### optimize
optimize (int): The level of graph optimization. Defaults to `GraphOptimizer.ALL_OPTIMIZE`

The optimizer levels are listed below.
- `NO_OPTIMIZE`
    No optimization is performed
- `FOLD_BUFFER`
    Only constant folding is performed
- `FUSE_BN`
    Fuse batch normalization with convolution and fully connected layers
- `COMMON_OPTIMIZE`
    Common optimizations like consecutive transpose/reshape/slice and no-op fusion/elimination
- `BRANCH_OPTIMIZE`
    Enable elementwise passthrough passes which apply to branch nodes
- `BRANCH_OPTIMIZE_EXTENDED`
    Enable the extended version of elementwise passthrough passes which apply to branch nodes
- `ALL_OPTIMIZE`
    Full optimization

It is often used in layerwise result comparison. Generally speaking, the more optimizations enabled, the less number of nodes that are shared between the PyTorch model and the TFLite model.

### group_conv_rewrite
group_conv_rewrite (bool): Rewriting for group convolution. Defaults to False

You have to set it to `True` if you want to run the models on standard TFLite backends when you have group convolutions in your model.

### tflite_micro_rewrite
tflite_micro_rewrite (bool): Rewriting for running on TFLite-micro. Defaults to False

There are specific constraints for running on TFLite Micro, such as the number of input tensors of the `CONCATENATION` operator cannot exceed 10. When you have such nodes (especially when you have group convolution), please set it to `True`.